In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
data_path = r'data'
filename = 'Lyrics_FrankSinatra.txt'
filepath = os.sep.join([data_path, filename])
checkpoint_path = r'checkpoints'

text = ''
with open(filepath, encoding='utf-8') as f:
    text = f.read()

print(f"len(text) = {len(text)}")
print("Example:", text[:50])

len(text) = 705123
Example: fly me to the moon let me play among the stars and


In [3]:
vocab = sorted(set(text))
print(f"len(vocab) = {len(vocab)}")

len(vocab) = 27


In [4]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(),
                                             invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [5]:
sequence_len = 100
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [6]:
class LyricsGeneratorModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=sequence_len)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x

In [7]:
EPOCHS = 80

# Load model
model = LyricsGeneratorModel(vocab_size=vocab_size,
                            embedding_dim=embedding_dim,
                            rnn_units=rnn_units)
model.build((None, 100))
model.load_weights(f"model_lower_char_{EPOCHS}_epochs_weights.h5")

In [8]:
# Create a mask to prevent "[UNK]" from being generated.
skip_ids = ids_from_chars(['[UNK]'])[:, None]
sparse_mask = tf.SparseTensor(
    # Put a -inf at each bad index.
    values=[-float('inf')]*len(skip_ids),
    indices=skip_ids,
    # Match the shape to the vocabulary
    dense_shape=[len(ids_from_chars.get_vocabulary())])
prediction_mask = tf.sparse.to_dense(sparse_mask)

def generate_one_step(inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = model(inputs=input_ids, states=states,
                                          return_state=True, training=False)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/1.0
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [9]:
start = time.time()
states = None
next_char = tf.constant(['life'])
result = [next_char]

for n in range(1000):
  next_char, states = generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

life begin so open up your heart and let this fool rush inhey drink up all you people order anything you see and have fun you happy people when the new heaven and earth shall own thy eyes those ill remember april and ill smiles posing i should fall in love with you evry day the thrill is always new evry day the morning sparkles spang at the sky now ive met miss jones and well keep on meeting till we die miss jones and id always ask for you amor meet up and down my spine the same old witchcrffa kiss the mist of day never never to know how i love you if i loved you brief instrumental soon youd leave me oh my dindi oh home not quictly cry dont night thats you and me is us with your heart it isnt smart it is no disfrceling for the gal who got that magic for you my heart got those jooms at tweet the birds that flew it say is this the day i alove all or its open and your favorite scone i cant have an echomain things to be seen when the wind is greenin in julets down or sunny weather the outl